# Setup

```
pip install git+https://github.com/scverse/spatialdata.git@main
pip install git+https://github.com/scverse/spatialdata-io.git@main
```

This is the stuff you have to edit; the rest of the sections can run as-is after you've set the needed parameters.

---

`coord_suffix` must align with 
* the sub-directory (corresponds to region, e.g., "mucosa") under the `dir_coord` directory where the Xenium Explorer-exported selection files are stored, and
* the suffixes of the coordinate selection files (see file naming conventions below).
  
The `AnnData` objects created will have this suffix as well (e.g., `Uninflamed-50452A_mucosa.h5ad`).

---

Selection files should be named by this convention:
`<library_id>_<coord_suffix>.csv`.

For example, if `dir_coord` is `.../coordinates/mucosa`, the mucosa selection file for sample 50452A should be under `.../coordinates/mucosa/50452A_mucosa.csv`. 

More specifically, if the coordinates directory is under `/mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library/outputs/TUQ97N/nebraska/coordinates`, and the selection region is "mucosa,"`dir_coord` should be `/mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library/outputs/TUQ97N/nebraska/coordinates/mucosa`, and the full file path for this sample would be `/mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library/outputs/TUQ97N/nebraska/coordinates/mucosa/50452A_mucosa.csv`.

---

**As with any other file naming schema, suffixes/directory names should not any special characters other than underscores (`_`) (no periods, dashes, spaces, etc.).**

N.B. In the above explanation, `library_id` refers to library/original sample ID without condition (e.g., "50452A", not "Uninflamed-50452A" like in other places). Remember that `coord_suffix` should also be the name of the parent directory of the coordinate file. I include this information in both the directory and file name to prevent mix-ups should files be moved or placed in the wrong folder.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import re
import math
import functools
import traceback
import anndata
import scanpy as sc
import numpy as np
import pandas as pd
import corescpy as cr

# Main
write_object = True  # change to True when you're ready to save objects
overwrite = False  # overwrite if already exists?
regions = ["mucosa", "serosa", "myenteric_plexus", "submucosa",
           "smc_longitudinal", "smc_circular"]

# Process Options
panel_id = "TUQ97N"  # Xenium panel ID
show_cols = [cr.pp.COL_SUBJECT]  # only change if not TUQ97N
cso = cr.pp.COL_SAMPLE_ID_O  # original library ID; change if not TUQ97N
col_sample = cr.pp.COL_SAMPLE_ID  # object sample ID; change if not TUQ97N
# libs = [  # sample IDs from patients for whom we have all conditions
#     "50452A", "50452B", "50452C",  # old segmentation
#     "50006A", "50006B", "50006C",  # rest are new segmentation
#     "50217A", "50217B", "50217C",
#     "50336B", "50336C", "50336A",
#     "50403A2", "50403B", "50403C1"
# ]  # excludes low-quality sample/condition replicates 50403A1 & 50403C2
libs = ["50006A", "50006B", "50217A", "50217B", "50336B", "50336C",
        "50403B", "50403A2"]  # just inflamed/uninflamed (no strictures)
# libs = None  # to run all available samples
input_suffix = ""  # in case want to crop objects with some suffix
# due to creation of a subsidiary object, e.g., for
# "Stricture-50452C_downsampled.h5ad"
# input_suffix would be "_downsampled". For "main" objects, input_suffix=""
plot = True  # could slow process down if large samples/cropped area

# Files & Directories
direc = "/mnt/cho_lab/bbdata2/"  # mounted NFS with data
dir_entry = "/mnt/cho_lab/disk2"  # Spark writeable data directory
mdf = str("/mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library/"
          "samples.csv")  # metadata file path (for now; will soon be on NFS)
dir_writeable = os.path.join(
    dir_entry, f"elizabeth/data/shared-xenium-library")  # where objects are
out_dir = os.path.join(
    dir_writeable, f"outputs/{panel_id}/nebraska")  # object output directory

#  Your Folders
out_new = os.path.join(
    dir_entry,
    f"{os.getlogin()}/data/shared-xenium-library/outputs/{panel_id}/nebraska")

# Constants (Shouldn't Need Edits Unless Extreme Process Changes)
dir_data = os.path.join(direc, f"outputs/{panel_id}")
col_obj = cr.pp.COL_OBJECT
# sc.logging.print_versions()

# Find Files & Load Metadata

Allows us to find the object IDs (e.g., for TUQ97N, object IDs are in the format <condition (Inflamed/Uninflamed/Stricture)><block_id>) corresponding to the sample IDs.

In [ ]:
files = functools.reduce(lambda i, j: i + j, [[os.path.join(
    run, i) for i in os.listdir(os.path.join(
        dir_data, run))] for run in os.listdir(dir_data)])  # all data paths
os.makedirs(out_dir, exist_ok=True)  # make output directory if needed
metadata = cr.pp.get_metadata_cho(direc, mdf, panel_id=panel_id, samples=libs)
metadata[show_cols]

# Subset Data by Coordinate Files & Write Cropped Objects

Subset the data by coordinates (`corescpy` can use Xenium Explorer-exported manual selection files to get those coordinates) and then write the cropped objects to `out_dir/<coord_suffix>`.

In [ ]:
%%time

for s in libs:  # iterate samples
    print(f"\n\n{'=' * 80}\n{s}\n{'=' * 80}\n\n")
    fff = os.path.join(dir_data, np.array(files)[np.where([
        s == os.path.basename(x).split("__")[2].split("-")[0]
        for x in files])[0][0]])  # sample's Xenium data directory path
    lib = metadata.reset_index().set_index(cso).loc[s][col_sample]
    file_obj_proc = os.path.join(out_dir, f"{lib}{input_suffix}.h5ad")
    self = cr.Spatial(fff, library_id=lib)  # load original data
    self.update_from_h5ad(file_obj_proc)  # update with processed object
    for coord_suffix in regions:
        print(f"\n\n\t\t{'*' * 40}\n{s}\n\t\t{'*' * 40}\n\n")
        dir_coord = os.path.join(
            out_new, "coordinates", coord_suffix)  # coordinates (NFS soon?)
        out = os.path.join(out_new, "objects_cropped", coord_suffix)  # path
        os.makedirs(out, exist_ok=True)  # make sub-directory for new objects?
        file_coord = os.path.join(dir_coord, s + f"_{coord_suffix}.csv")
        file_obj_crop = os.path.join(out, f"{lib}_{coord_suffix}.h5ad")
        if overwrite is False and os.path.exists(file_obj_crop):
            print(f"*** Subset {file_obj_crop} already exists")
            continue
        if not os.path.exists(file_coord):
            print(f"*** Coordinate file {file_coord} doesn't exist")
            continue
        print(f"\n\nData: {fff}\nObject: {file_obj_proc}"
              f"\nCoordinates: {file_coord}\nOuput: {file_obj_crop}")
        try:
            sdata = self.crop(file_coord)  # crop data to coordinates
            if plot:
                try:
                    sdata.pl.render_labels("cell_labels").pl.show()  # plot
                except Exception:
                    pass
            self.adata = sdata
            if write_object is True:
                self.write(file_obj_crop)  # write cropped
        except Exception:
            print(traceback.format_exc(), f"Cropping \n\n{s} failed!")

In [27]:
sdata = self.crop(file_coord)  # crop data to coordinates
sdata.pl.render_labels("cell_labels").pl.show()  # plot
self.adata = sdata
self.write(file_obj_crop)  # write cropped

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
element = self.adata.shapes["cell_circles"]
element

In [ ]:
buffered_df = element.copy()

In [ ]:
adata

In [ ]:
from spatialdata import to_polygons

element = self.adata.shape["cell_circles"]
buffered = to_polygons(element) if ShapesModel.RADIUS_KEY in element.columns else element

# Workspace

In [ ]:
# n_cells = {}
# for s in libs:  # iterate samples
#     print(f"\n\n{'*' * 40}\n{s}\n{'*' * 40}\n\n")
#     fff = os.path.join(dir_data, np.array(files)[np.where([
#         s == os.path.basename(x).split("__")[2].split("-")[0]
#         for x in files])[0][0]])  # sample's Xenium data directory path
#     lib = metadata.reset_index().set_index(cso).loc[s][col_sample]
#     file_obj_proc = os.path.join(out_dir, f"{lib}{input_suffix}.h5ad")
#     self = cr.Spatial(fff, library_id=lib)  # load original data
#     n_obs = self.rna.obs.shape[0]
#     self.update_from_h5ad(file_obj_proc)  # update with processed object)
#     self.rna.obs.loc[:, "n_obs_raw"] = n_obs
#     self.write(file_obj_proc)
#     n_cells[s] = pd.Series([n_obs, self.rna.obs.shape[0]], index=pd.Index([
#         "Raw", "Processed"], name="Source"))
# n_cells = pd.concat(n_cells).unstack("Source")
# n_cells.to_excel("/home/elizabeth/elizabeth/projects/senescence/metadata.xlsx")
n_cells